In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os, glob
import shutil
import cv2 as cv

C:\Users\Idris\Anaconda3\lib\site-packages\skimage\viewer\utils\core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.regularizers import l1, l2

In [3]:
# Sorting the files into correct folders based on name on my local computer

#for folder in ('train', 'test'):
folder = 'train'
unsorted_img_paths = glob.glob('I:/count_fingers_cnn/fingers_2/{}/*.png'.format(folder))
for img_path in unsorted_img_paths:
    shutil.move(img_path, 'I:/count_fingers_cnn/fingers_2/{}/'.format(folder)+img_path[-5:-4])



In [10]:
img_size = 128
num_classes = 12
batch_size = 96
epochs = 6

model = Sequential()
model.add(Conv2D(64, kernel_size=(2,2), activation='elu', padding='same', input_shape=(img_size, img_size, 1)))

model.add(Conv2D(64, kernel_size=(3,3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(num_classes, activation='softmax'))


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 64)      320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 64)       

In [12]:
# Helper Functions


from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# turn an array of prediction chances into a single value of the highest probability prediction
def decode_prediction(pred): 
    #classes = ['0','1','2','3','4','5']
    classes = ['0L', '0R', '1L', '1R', '2L', '2R', '3L', '3R', '4L', '4R', '5L', '5R']
    pred = list(pred)
    return classes[pred.index(max(pred))]

# same as above but for multiple predictions of different hands at once
def decode_predictions(preds):
    return [decode_prediction(pred) for pred in preds]

# predict with a model from an image variable
def predict_from_image(model, img):
    img_arr = np.array([img_to_array(img)])
    preds = model.predict(img_arr)
    output = decode_prediction(preds[0])
    print(preds[0])
    return output

# predict with a model from an image file path
def predict_from_image_path(model, img_path):
    img = load_img(img_path, target_size=(img_size, img_size), color_mode='grayscale')
    return predict_from_image(model, img)

# convert a cv image/array to a PIL image
def convert_cv_to_pil(img_cv):
    #img_cv = cv.cvtColor(img_cv, cv.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_cv)
    return img_pil

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Getting data from the directories

train_data_gen = ImageDataGenerator(
                                    #horizontal_flip = True,
                                    width_shift_range = 0.05,
                                    height_shift_range = 0.05,
                                    zoom_range = 0.05,
                                    rotation_range = 20,
                                    #validation_split = 0.2
                                    )

val_data_gen = ImageDataGenerator() 

train_gen = train_data_gen.flow_from_directory(directory='I:/count_fingers_cnn/fingers/train', # CHANGE TO 'fingers' DIRECTORY
                                              target_size=(img_size,img_size),
                                              color_mode='grayscale',
                                              class_mode='categorical',
                                              batch_size=batch_size
                                              #subset = 'training'
                                              )


val_gen = val_data_gen.flow_from_directory(directory='I:/count_fingers_cnn/fingers/test',  # CHANGE TO 'fingers' DIRECTORY
                                           target_size=(img_size,img_size), 
                                           color_mode='grayscale',
                                           class_mode='categorical',
                                           batch_size=batch_size
                                           #subset = 'validation'
                                            )

Found 18000 images belonging to 12 classes.
Found 3600 images belonging to 12 classes.


In [14]:
# Compiling the model

model.compile(loss='categorical_crossentropy', 
             optimizer='adam',
             metrics=['accuracy'])

In [15]:
# Fitting the model with the parameters

model.fit(train_gen,
          epochs=epochs,
          validation_data=val_gen)

Epoch 1/6
188/188 [==============================] - 53s 282ms/step - loss: 3.1821 - acc: 0.4849 - val_loss: 0.3251 - val_acc: 0.8667
Epoch 2/6
188/188 [==============================] - 49s 262ms/step - loss: 0.1870 - acc: 0.9334 - val_loss: 0.0854 - val_acc: 0.9689
Epoch 3/6
188/188 [==============================] - 50s 266ms/step - loss: 0.0836 - acc: 0.9722 - val_loss: 0.0229 - val_acc: 0.9928
Epoch 4/6
188/188 [==============================] - 51s 269ms/step - loss: 0.0466 - acc: 0.9850 - val_loss: 0.0123 - val_acc: 0.9958
Epoch 5/6
188/188 [==============================] - 53s 284ms/step - loss: 0.0382 - acc: 0.9877 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 6/6
188/188 [==============================] - 51s 273ms/step - loss: 0.0320 - acc: 0.9899 - val_loss: 7.9302e-04 - val_acc: 1.0000


In [19]:
# Testing the model on different hands from files on my local computer


#random_img_path = '../input/fingers-formatted/test/5R/01077f28-4d49-4fb0-bcdf-74a11d41214d_5R.png'
#print(predict_from_image_path(model, random_img_path))

#test_finger_path = 'I:/count_fingers_cnn/my_fingers/2R.jpg'
#test_finger_path = 'I:/count_fingers_cnn/fingers/test/4L/0a678a65-ec44-4ff4-a92e-280d5183e98b_4L.png'
#test_finger_path = 'I:/count_fingers_cnn/fingers_2/train/1/0_1.png'

#test_finger_img = load_img(test_finger_path, target_size=(img_size, img_size), color_mode = 'grayscale')
#test_finger_img = prep_image_from_path(test_finger_path)

#test_finger_img.show()

# testing model from saved file
#model2 = tf.keras.models.load_model('I:/count_fingers_cnn/v2-desktop/model.pb')
#print(predict_from_image_path(model2, test_finger_path))


[0.0000000e+00 0.0000000e+00 7.3246123e-32 2.9341125e-36 1.7067546e-18
 7.8358065e-23 2.6730524e-07 5.7289306e-15 9.9999976e-01 2.3239431e-11
 2.1264503e-10 8.3831232e-15]
4L


In [18]:
# Saving the model to a file

#model.save('I:/count_fingers_cnn/v2-desktop/model.pb')
#model_from_file = tf.keras.models.load_model('I:/count_fingers_cnn/v2-desktop/model.pb')